In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os
from pathlib import Path

from requests import get
import pandas as pd
import numpy as np

np.random.seed(0)

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

import logging

logging.basicConfig(level=logging.WARN)


<IPython.core.display.Javascript object>

In [3]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import (
    Conv1D,
    SpatialDropout1D,
    LocallyConnected1D,
    Dense,
    Reshape,
    MaxPooling1D,
    BatchNormalization,
    Activation,
    LayerNormalization,
)

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

from tensorflow_addons.activations import mish
from tensorflow_addons.optimizers import RectifiedAdam, Lookahead
from tensorflow_addons.layers import WeightNormalization

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder


from itertools import repeat
from concurrent.futures import ProcessPoolExecutor as PoolExecutor

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline


<IPython.core.display.Javascript object>

In [4]:
def word_to_np_array(word, cut_length):
    result = np.zeros(cut_length, dtype="uint8")
    for i, letter in enumerate(word[:cut_length]):
        result[i] = ord(letter)
    return result

<IPython.core.display.Javascript object>

In [5]:
def line_to_img(line, cut_length):
    result = np.zeros((line.shape[0], cut_length), dtype="uint8")
    for i in range(line.shape[0]):
        result[i] = word_to_np_array(line[i], cut_length)
    return result

<IPython.core.display.Javascript object>

In [6]:
def do_parallel_numpy(map_func, iter_params, constant_params=None):
    repeated_params = (
        [] if constant_params is None else list(map(repeat, constant_params))
    )
    results = None
    with PoolExecutor() as executor:
        results = np.stack(
            list(executor.map(map_func, *iter_params, *repeated_params)), axis=0
        )
    return results

<IPython.core.display.Javascript object>

In [7]:
def download(url, out, force=False, verify=True):
    out.parent.mkdir(parents=True, exist_ok=True)
    if force and out.exists():
        print(f"Removing file at {str(out)}")
        out.unlink()

    if out.exists():
        print("File already exists.")
        return
    print(f"Downloading {url} at {str(out)} ...")
    # open in binary mode
    with out.open(mode="wb") as file:
        # get request
        response = get(url, verify=verify)
        for chunk in response.iter_content(100000):
            # write to file
            file.write(chunk)


<IPython.core.display.Javascript object>

In [8]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if "loss" in s and "val" not in s]
    val_loss_list = [s for s in history.history.keys() if "loss" in s and "val" in s]
    acc_list = [s for s in history.history.keys() if "AUC" in s and "val" not in s]
    val_acc_list = [s for s in history.history.keys() if "AUC" in s and "val" in s]

    if len(loss_list) == 0:
        print("Loss is missing in history")
        return

    ## As loss always exists
    epochs = range(1, len(history.history[loss_list[0]]) + 1)

    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(
            epochs,
            history.history[l],
            "b",
            label="Training loss ("
            + str(str(format(history.history[l][-1], ".5f")) + ")"),
        )
    for l in val_loss_list:
        plt.plot(
            epochs,
            history.history[l],
            "g",
            label="Validation loss ("
            + str(str(format(history.history[l][-1], ".5f")) + ")"),
        )

    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()

    plt.show()

<IPython.core.display.Javascript object>

In [9]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
url_test = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test"

dataset_name = "census-income"
out = Path(os.getcwd() + "/data/" + dataset_name + ".csv")
out_test = Path(os.getcwd() + "/data/" + dataset_name + "_test.csv")

download(url, out, force=False)
download(url_test, out_test, force=False)


File already exists.
File already exists.


<IPython.core.display.Javascript object>

In [10]:
cols = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education-num",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital-gain",
    "capital-loss",
    "hours-per-week",
    "native-country",
    "target",
]

<IPython.core.display.Javascript object>

In [11]:
train = pd.read_csv(out, names=cols)
test = pd.read_csv(out_test, names=cols, skiprows=2)
target = "target"

train[target] = train[target].str.strip()
# Test has . in label, let's clean it
test[target] = test[target].str.strip().str.strip(".")

<IPython.core.display.Javascript object>

In [12]:
target_encoder = LabelEncoder()

<IPython.core.display.Javascript object>

In [13]:
train[target] = target_encoder.fit_transform(train[target].values.reshape(-1))
test[target] = target_encoder.transform(test[target].values.reshape(-1))

<IPython.core.display.Javascript object>

In [14]:
used_columns = list(set(train.columns.tolist()) - set([target]) - set(["Set"]))
used_columns

['capital-gain',
 'occupation',
 'race',
 'education-num',
 'capital-loss',
 'workclass',
 'age',
 'education',
 'sex',
 'hours-per-week',
 'native-country',
 'relationship',
 'fnlwgt',
 'marital-status']

<IPython.core.display.Javascript object>

In [15]:
train["Set"] = np.random.choice(
    ["train", "valid"], p=[0.8, 0.2], size=(train.shape[0],)
)

train_indices = train[train.Set == "train"].index
valid_indices = train[train.Set == "valid"].index

X_train = np.char.strip(train[used_columns].values[train_indices].astype("str"))
X_valid = np.char.strip(train[used_columns].values[valid_indices].astype("str"))

y_train = train[target].values[train_indices]
y_valid = train[target].values[valid_indices]

# Test here should be ignored for training, only purpose is benching with paper values
X_test = np.char.strip(test[used_columns].values.astype("str"))
y_test = test[target].values

del train, test, train_indices, valid_indices

<IPython.core.display.Javascript object>

In [16]:
X_train.shape

(26072, 14)

<IPython.core.display.Javascript object>

In [17]:
INPUT_DIM = X_train.shape[1]
NB_CHANNELS = np.vectorize(len)(X_train).max()
NB_CHANNELS

26

<IPython.core.display.Javascript object>

In [18]:
line_to_img(X_train[0], 27).shape

(14, 27)

<IPython.core.display.Javascript object>

In [19]:
X_train_preproc = do_parallel_numpy(line_to_img, [X_train], [NB_CHANNELS])
X_valid_preproc = do_parallel_numpy(line_to_img, [X_valid], [NB_CHANNELS])
X_test_preproc = do_parallel_numpy(line_to_img, [X_test], [NB_CHANNELS])

<IPython.core.display.Javascript object>

Y_train_preproc = to_categorical(y_train)
Y_valid_preproc = to_categorical(y_valid)
Y_test_preproc = to_categorical(y_test)

In [20]:
def build_model(input_dim, nb_channels, conv_dim=[], lconv_dim=[]):
    activation = mish
    optimizer = Lookahead(RectifiedAdam(1e-3), sync_period=6, slow_step_size=0.5)

    input_layer = Input(shape=(input_dim, nb_channels), name="input")
    x_layer = input_layer
    for i, conv_layer in enumerate(conv_dim):
        name = f"block_conv_{i}_"
        x_layer = Conv1D(
            filters=conv_layer,
            padding="valid",
            kernel_size=1,
            strides=1,
            name=name + "conv",
            use_bias=False,
            activation=None,
        )(x_layer)
        x_layer = BatchNormalization(name=name + "nb")(x_layer)
        x_layer = Activation(activation, name=name + "activation")(x_layer)

    for i, lconv_layer in enumerate(lconv_dim):
        name = f"block_lconv_{i}_"
        x_layer = LocallyConnected1D(
            filters=lconv_layer,
            padding="valid",
            kernel_size=1,
            strides=1,
            name=name + "lconv",
            use_bias=False,
            activation=None,
        )(x_layer)
        x_layer = BatchNormalization(name=name + "nb")(x_layer)
        x_layer = Activation(activation, name=name + "activation")(x_layer)

    nb_filters = lconv_dim[-1] if len(lconv_dim) > 0 else conv_dim[-1]
    x_layer = Reshape((input_dim * nb_filters,), name="reshape")(x_layer)
    x_layer = Dense(1, activation="sigmoid", name="output")(x_layer)

    model = Model(inputs=[input_layer], outputs=[x_layer], name="first_model")
    model.compile(loss="binary_crossentropy", optimizer=optimizer)

    return model

<IPython.core.display.Javascript object>

In [21]:
model = build_model(INPUT_DIM, NB_CHANNELS, conv_dim=[64], lconv_dim=[128, 64, 32])

/work/.cache/poetry/thc-net-KQLMmzPP-py3.7/lib/python3.7/site-packages/tensorflow_addons/options.py:47: RuntimeWarning: Traceback (most recent call last):
  File "/work/.cache/poetry/thc-net-KQLMmzPP-py3.7/lib/python3.7/site-packages/tensorflow_addons/activations/mish.py", line 44, in mish
    return _mish_custom_op(x)
  File "/work/.cache/poetry/thc-net-KQLMmzPP-py3.7/lib/python3.7/site-packages/tensorflow_addons/activations/mish.py", line 59, in _mish_custom_op
    return _activation_so.ops.addons_mish(x)
  File "/work/.cache/poetry/thc-net-KQLMmzPP-py3.7/lib/python3.7/site-packages/tensorflow_addons/utils/resource_loader.py", line 64, in ops
    self._ops = tf.load_op_library(get_path_to_datafile(self.relative_path))
  File "/work/.cache/poetry/thc-net-KQLMmzPP-py3.7/lib/python3.7/site-packages/tensorflow/python/framework/load_library.py", line 58, in load_op_library
    lib_handle = py_tf.TF_LoadLibrary(library_filename)
tensorflow.python.framework.errors_impl.NotFoundError: /work/

<IPython.core.display.Javascript object>

In [22]:
model.summary()

Model: "first_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 14, 26)]          0         
_________________________________________________________________
block_conv_0_conv (Conv1D)   (None, 14, 64)            1664      
_________________________________________________________________
block_conv_0_nb (BatchNormal (None, 14, 64)            256       
_________________________________________________________________
block_conv_0_activation (Act (None, 14, 64)            0         
_________________________________________________________________
block_lconv_0_lconv (Locally (None, 14, 128)           114688    
_________________________________________________________________
block_lconv_0_nb (BatchNorma (None, 14, 128)           512       
_________________________________________________________________
block_lconv_0_activation (Ac (None, 14, 128)           

<IPython.core.display.Javascript object>

In [23]:
y_train.shape

(26072,)

<IPython.core.display.Javascript object>

In [24]:
history = model.fit(
    X_train_preproc,
    y_train.reshape(-1, 1),
    epochs=2000,
    batch_size=1024,
    validation_data=(X_valid_preproc, y_valid.reshape(-1, 1),),
    verbose=1,
    callbacks=[EarlyStopping(monitor="val_loss", patience=50, verbose=1)],
)

Epoch 1/2000


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node first_model/block_conv_0_conv/conv1d (defined at <ipython-input-24-aafb47b730f0>:8) ]] [Op:__inference_train_function_6677]

Function call stack:
train_function


<IPython.core.display.Javascript object>

In [ ]:
plot_history(history)

In [ ]:
model_auc = roc_auc_score(
    y_true=y_valid, y_score=model.predict(X_valid_preproc).reshape(-1),
)
model_auc

In [ ]:
# 0.9267169419995929 128  =[64], lconv_dim=[128, 64, 32

In [ ]:
model_auc = roc_auc_score(
    y_true=y_test, y_score=model.predict(X_test_preproc).reshape(-1),
)
model_auc